#### Translation von WI Data in EN

Nutzung von GoogleTrans Sprachmodell

Hinweis:
Wurde ausprobiert, um Nutzung von kostenloser Google API zu machen

Problem:
-->Nur begrenzte Zeichenanzahl
-->Nur begrenzte Methoden (Check der Sprache ist nicht frei)

    Daher mit nltk Abstracts gesplittet, um die Sätze einzeln übersetztn zu können --> Tests waren erfolgreich, es funktioniert aber nur bis ca 100 Artikel

TODO hier (Wenn weiter verfolgt werden soll):
-->ändern der Bibliothek zu google Translate/ das macht aber Probleme beim Laden, löst sich vermutlich wenn man bezahlt
-->Struktur ordentlicher, Übersetzung von Titeln und Abstracts trennen
--> Ergänzen von try/except


In [346]:
#https://cloud.google.com/translate/docs/
import os
#from google.cloud import translate

try:
    # Setze den API-Schlüssel als Umgebungsvariable
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path/to/api-key.json'

    # Erstelle ein Translate-Client-Objekt
    translate_client = translate.Client()

except:
    print("Error: creating translate client")

Error creating translate client


In [326]:
#Import von Daten
import pandas as pd

publication = pd.read_csv('wi_df_final_clean_keyword.csv',encoding='utf8')


In [327]:
publication.head()

,id,cfTitle,cfUri,keywords,doi,srcAuthors,authors,cfAbstr,publYear,eid,data_source,log,result_flag,error,error_doi,keyword_clean
0,10101316,Configuring Reference Models - An Integrated A...,NaN,"['Configuration', 'Decision support', 'Integra...",NaN,"Knackstedt R, Janiesch C, Rieke T","Knackstedt, Ralf;Janiesch, Christian;Rieke, To...","Reference models are of normative, universal n...",2006,2-s2.0-67649915340,scopus-api,{'log_title': 'configuring reference models - ...,True,NaN,NaN,"['Configuration, Decision support, Integration..."
1,10105507,Konnektionistische Tourenplanung für die dezen...,NaN,NaN,NaN,"Pietsch W, Teubner RA",NaN,Die Aufstellung effizienter Tourenpl&auml;ne s...,1992,NaN,NaN,NaN,False,NaN,NaN,NaN
2,10971967,Cryptocoin-Schulden – Haftung und Risikovertei...,NaN,NaN,NaN,Pesch,NaN,NaN,2017,NaN,NaN,NaN,False,NaN,NaN,NaN
3,10976734,Benefits and Perils of Virtual Modes of Organi...,NaN,NaN,10.4324/9781315601915,"Schellhammer Stefan, Klein Stefan","Schellhammer, Stefan;Klein, Stefan",NaN,2016,2-s2.0-85062635276,scopus-api,{'log_title': 'benefits and perils of virtual ...,True,NaN,NaN,NaN
4,10982470,Dynamic Decision Making in Energy Systems with...,NaN,"['Energy storage', 'Sequential decision making...",10.1007/978-3-319-51795-7_6,"Meisel S, Powell W","Meisel, Stephan;Powell, Warren B.",<html />,2017,2-s2.0-85048159112,scopus-api,{'log_title': 'dynamic decision making in ener...,True,NaN,NaN,"['Energy storage, Sequential decision making, ..."


In [255]:
#Parameter von DeepL
from deep_translator import GoogleTranslator
import nltk

def translate_publication_wi(publication):
    """
    Funktion durchläuft alle Zeile des Pandas DF und schaut für die jeweiligen Einträge ob sie auf englisch vorleigen. Wenn dies nicht der Fall ist, werden die auf englisch übersetzt
    --> Datei wird nach jedem Übersetzungsvorgang exportiert, das hat den Grund das nicht ein Fehler auftritt und damit der Fortschritt umsost ist
    :param publication:
    :return:
    """
    #print(type(publication))
    publication_en = publication


    # Iterate over the rows of the DataFrame
    for index, row in publication_en.iterrows():

        actual_title = str(row['cfTitle'])
        actual_abstr = str(row['cfAbstr'])

        if(actual_title != ("" or 'nan')):
            result = translator(actual_title)
            translated_titel = result
            #Insert translated value
            publication_en.loc[index, 'cfTitle'] = str(translated_titel)

        if(str(actual_abstr) != ("" or 'nan')):
            #More than 5000 Words, split and translate seperate
            translated_abstr_split = splitText(str(actual_abstr))
            translated_abstr = join_text(translated_abstr_split)
            #Insert translated value
            publication_en.loc[index, 'cfAbstr'] = str(translated_abstr)
            #print(translated_abstr)


    #Save translation
    #csv_exporter(publication_en)
    return publication_en


def translator(input):
    result = GoogleTranslator(source='auto', target='en').translate(input)
    return result

def csv_exporter(dataframe):
    dataframe.to_csv(r'.\wi_df_translated\wi_df_translated_googleTrans.csv', index=False)
    pass


#Methods to bypass maximum string length
def splitText(actual_str):

    actual_str = nltk.tokenize.sent_tokenize(actual_str)
    translated = []
    for sentence in actual_str:
        result = translator(sentence) #send each sentence to translator
        translated.append(result)
    return translated

def join_text(input):
    trennzeichen = ' '
    ergebnis = trennzeichen.join(input)
    return ergebnis

#### Google Translate Keywords

In [203]:
#### Function to translate Keywords
def translate_publication_keywords_wi(publication):
    """
    The function runs through all lines of the panda DF and looks for the respective entries to see whether they are in English. If this is not the case, they will be translated into English
    :param publication:
    :return:
    """
    publication_keywords = publication

    target = 'en'

    for index, row in publication_keywords.iterrows():
        data = row['keyword_clean']
        if(type(data) == str):
            actual_Data = eval(data)
            #for actual_str in actual_Data:  (If each element is to be translated individually)
            actual_language = check_language(actual_Data)

            if(actual_language != 'en'):
                translation = translate_client.translate(actual_Data, target_language=target)
                translated_text = translation['translatedText']
                publication_keywords.loc[index, 'keywords'] = str(translated_text)

    return publication_keywords


def csv_exporter(dataframe):
    dataframe.to_csv(r'.\wi_df_translated\wi_df_translated_final.csv', index=False)
    pass

In [203]:
def check_language(text):
    result = translate_client.detect_language(text)
    return result['language']
    pass


In [338]:
#First, titles and abstracts are translated
translated_title_abstr = translate_publication_wi(publication)
#Translate keywords
all_translated =translate_publication_keywords_wi(translated_title_abstr)
#Export a dataset with all entries translated to English
csv_exporter(all_translated)